<a href="https://www.kaggle.com/code/turtlejaywang/vox2sdf?scriptVersionId=218424939" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Vox2SDF
This is the demo of Vox2SDF.
Vox2SDF is a model that convert voxel object to DeepSDF representation.

## Clone things from git

In [1]:
!git clone --recursive https://github.com/TurtleJayWang/Vox2SDF.git

Cloning into 'Vox2SDF'...
remote: Enumerating objects: 160, done.
remote: Counting objects: 100% (160/160), done.
remote: Compressing objects: 100% (118/118), done.
remote: Total 160 (delta 91), reused 107 (delta 40), pack-reused 0 (from 0)
Receiving objects: 100% (160/160), 230.41 KiB | 4.11 MiB/s, done.
Resolving deltas: 100% (91/91), done.
Submodule 'dataset' (https://github.com/TurtleJayWang/VoxelSDFDataset.git) registered for path 'dataset'
Cloning into '/kaggle/working/Vox2SDF/dataset'...
remote: Enumerating objects: 95, done.        
remote: Counting objects: 100% (95/95), done.        
remote: Compressing objects: 100% (66/66), done.        
remote: Total 95 (delta 45), reused 71 (delta 24), pack-reused 0 (from 0)        
Receiving objects: 100% (95/95), 22.50 KiB | 11.25 MiB/s, done.
Resolving deltas: 100% (45/45), done.
Submodule path 'dataset': checked out 'a0951ebbbad88fa5578a6a3dd2780623bdb5c7dd'
Submodule 'thirdparty/cuda_voxelizer' (https://github.com/Forceflow/cuda_voxe

In [11]:
!cd Vox2SDF; git pull

remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 4 (delta 3), reused 4 (delta 3), pack-reused 0 (from 0)
Unpacking objects: 100% (4/4), 341 bytes | 341.00 KiB/s, done.
From https://github.com/TurtleJayWang/Vox2SDF
   f8748bc..9a3a70b  main       -> origin/main
Updating f8748bc..9a3a70b
Fast-forward
 Scripts/train.py | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)


## Download Data from Kaggle Dataset

In [3]:
import kagglehub
path = kagglehub.dataset_download("turtlejaywang/voxelsdfdataset")

# Configuration

In [4]:
import json
import os
config = {
    "train_epoch" : 10,
    "learning_rate" : 0.0001,
    "input_voxel_grid_size" : 64,
    "decoder_hidden_dimension" : 512,
    "decoder_num_hidden_layers" : 8,
    "latent_dimension" : 256,
    "num_sdf_samples" : 250000,
    "num_points_per_minor_batch" : 12500,
    "dataset_path" : os.path.join(path, "VoxelSDFDataset/data"),
    "cuda_voxelizer_path" : "thirdparty/cuda_voxelizer/cuda_voxelizer",
    "shapenet_pickle_name" : "shapenet_data.pkl",
    "shapenet_categories" : ["02747177"],
    "num_augment_data" : 4,
    "seperate_ratio" : [0.8, 0.2],
    "check_point_filename" : "checkpoint.pkl",
    "loss_filename" : "loss.pkl",
    "is_train" : 1,
    "is_validation" : 1
}
with open("Vox2SDF/Scripts/config.json", "w") as f:
    json.dump(config, f)

print("Complete configuration!")

Complete configuration!


# Train the model

In [16]:
!cd Vox2SDF; python Scripts/train.py

/bin/bash: line 1: cd: Vox2SDF: No such file or directory
python3: can't open file '/kaggle/working/Vox2SDF/checkpoints/Scripts/train.py': [Errno 2] No such file or directory


In [21]:
import os
import shutil
shutil.move('/kaggle/working/Vox2SDF/checkpoints/checkpoint.pkl', '/kaggle/working/Vox2SDF/checkpoint.pkl')
from IPython.display import FileLink

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/working/Vox2SDF/checkpoints/loss.pkl'

# The saved checkpoint

In [9]:
!kaggle datasets init -p /kaggle/working/Vox2SDF/checkpoints
!kaggle datasets create -p /kaggle/working/Vox2SDF/checkpoints

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 7, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 407, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.config/kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 7, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 407, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSErro

# Visualize(To Do)